# Fantasy Premier League (FPL) Advisor

# Purpose
The purpose of this Jupyter notebook is to help with the selection of team members for the [Fantasy Premier League](https://fantasy.premierleague.com/) (FPL) by attempting to forecast how many points players will earn. It accesses the FPL API to download up-to-date stats, provides visual analysis and uses linear optimisation to recommend a team with the maximum expected points to improve the performance of your current team.

If you are not familar with the Fantasy Permier League, you can watch this introduction:

<a href="http://www.youtube.com/watch?v=SV_F-cL8fC0" target="_blank"><img src="http://img.youtube.com/vi/SV_F-cL8fC0/0.jpg" 
alt="How to play FPL" width="600" height="400"/></a>

# Installation
To get started, run the following command to install all required dependencies.

In [ ]:
 !pip install -r requirements.txt

# Import requirements
Here we import all external and local modulues.

In [ ]:
import pandas as pd, re, datetime as dt, numpy as np, plotly.offline as py, cufflinks as cf, ipywidgets as widgets, os, sys
from ipywidgets import interact, fixed
from fplpandas import FPLPandas
from datadict.jupyter import DataDict

# Load local modules
sys.path.append(os.getcwd())
from optimiser import get_optimal_squad
from common import *
from jupyter import *

cf.go_offline()
pd.set_option('display.max_columns', 100)

# Set variables
This section sets all important global variables.

In [ ]:
fpl = FPLPandas()

# Load data dictionary
This section loads the data dictionary. The data dictionary contains default ordering of fields, for each field a description default format and mapping of API field names to more readable ones. It is used show data in a more user-friendly way.

In [ ]:
dd = DataDict(data_dict_file=f'./data_dictionary.csv')

# Load player data
This section loads the player data and stats from the following FPL API endpoint: https://fantasy.premierleague.com/api/bootstrap-static/ and returns it as a panda data frame. **This can take a few seconds** because for each player the full history for the current season is downloaded.

In [ ]:
players_raw, players_history_past_raw, players_history_raw, player_fixtures_raw = fpl.get_players()

Then we rename some columns so that they are easier to understand and uses the data dictionary to reorder them so that the more important ones are at the front.

In [ ]:
def get_news(row: pd.Series) -> str:
    '''Derives the text for the News column.'''
    if pd.isnull(row['news']) or row['news'] == '': return None
    date_part = '' if pd.isnull(row['news_added']) else ' ('+dt.datetime.strftime(dt.datetime.strptime(row['news_added'], '%Y-%m-%dT%H:%M:%S.%fZ'), '%d %b %Y')+')'
    return str(row['news'])+date_part

players = dd.remap(players_raw, data_set='player')
players['ICT Index'] = pd.to_numeric(players['ICT Index'])
players['Field Position'] = players['Field Position ID'].map(lambda x: position_by_type[x])
players['Current Cost'] = players['now_cost']/10
players['Minutes Percent'] = players['Minutes Played']/players['Minutes Played'].max()*100
players['News And Date'] = players.apply(lambda row: get_news(row), axis=1)
players['Percent Selected'] = pd.to_numeric(players['Percent Selected'])
players['Chance Avail This GW'] = players['Chance Avail This GW'].map(lambda x: x if not pd.isnull(x) else 100)
players['Chance Avail Next GW'] = players['Chance Avail Next GW'].map(lambda x: x if not pd.isnull(x) else 100)
players.index = players.index.rename('Player ID')
players = dd.reorder(players)
# Uncomment to view data: dd.display(players, head=10, excel_file='players.xlsx', stats=True)

In [ ]:
players_history_past = dd.remap(players_history_past_raw, data_set='players_history_past')
players_history_past.index = players_history_past.index.rename(['Player ID', 'Season'])
seasons = players_history_past.reset_index()['Season'].drop_duplicates().sort_values().values
current_season = seasons[-1:][0]
last_season = seasons[-2:-1][0]
# Uncomment to view data: dd.display(players_history_past, head=10, excel_file='players_history_past.xlsx', stats=True)

# Load data for completed games this season for all players
This section loads the data for completed games this season for each player from the following endpoint: https://fantasy.premierleague.com/api/element-summary/1/ and returns it as a panda data frame.

In [ ]:
players_history = dd.remap(players_history_raw, 'player_hist')
players_history.index.names = ['Player ID', 'Fixture ID']
players_history['Game Cost'] = players_history['value']/10
players_history['Game ICT Index'] = pd.to_numeric(players_history['Game ICT Index'])
# Uncomment to view data: dd.display(players_history, head=10, stats=True, excel_file='players_history.xlsx')

# Load team data

This section loads the team data and stats from the following endpoint: https://fantasy.premierleague.com/api/bootstrap-static/ and returns it as a panda data frame.

In [ ]:
teams_raw = fpl.get_teams()

He we rename some columns so that they are easier to understand and uses the data dictionary to reorder the columns so that the more important ones are at the front.

In [ ]:
teams = dd.remap(teams_raw, 'team')
teams.index = teams.index.rename('Team ID')
teams = teams.drop(columns=['Strength Attack Home',	'Strength Attack Away', 'Strength Defence Home', 'Strength Defence Away', 'Strength Overall Home', 'Strength Overall Away'])
# Uncomment to view data: dd.display(teams, head=10)

# Load fixture data

This section loads the fixture data and stats from the following endpoint: https://fantasy.premierleague.com/api/fixtures/ and returns it as a panda data frame.

In [ ]:
fixtures_raw = fpl.get_fixtures()

Here we rename some columns so that they are easier to understand and uses the data dictionary to reorder the columns so that the more important ones are at the front.

In [ ]:
fixtures = dd.remap(fixtures_raw.drop(columns=['stats', 'finished_provisional', 'provisional_start_time']), 'fixture')
fixtures.index = fixtures.index.rename('Fixture ID')
# Uncomment to view data: dd.display(fixtures, head=10)

# Create derived data
This section creates new dataset by combining the previously loaded ones.

## Players with team info

In [ ]:
player_teams = players.reset_index(drop=False).merge(teams, left_on='Player Team ID', right_on='Team ID').set_index('Player ID')
player_teams['Long Name'] = player_teams['First Name']+' '+player_teams['Last Name']
player_teams['Long Name and Team'] = player_teams['Long Name']+' ('+player_teams['Team Name']+')'
player_teams['Name and Short Team'] = player_teams['Name']+' ('+player_teams['Team Short Name']+')'
player_teams = dd.reorder(player_teams)
# Uncomment to view data: dd.display(player_teams, head=5)

## Fixtures with team info

In [ ]:
fixture_teams = fixtures.reset_index().merge(teams[['Team Name', 'Team Short Name']]
                               .rename(columns={'Team Name': 'Team Name Home', 'Team Short Name': 'Team Short Name Home'}), 
                               left_on='Home Team ID', 
                               right_on='Team ID')
fixture_teams = fixture_teams.merge(teams[['Team Name', 'Team Short Name']]
                                    .rename(columns={'Team Name': 'Team Name Away', 'Team Short Name': 'Team Short Name Away'}), 
                               left_on='Away Team ID', 
                               right_on='Team ID')
fixture_teams = dd.reorder(fixture_teams)
# Uncomment to view data: dd.display(fixture_teams, head=10)

## Player derived fields and metrics
The section below derives a few useful player attributes but most importantly, it calculates the total points earned by a player devided by his current cost. This is can be an indicator for whether the player is undervalued or overpriced.

In [ ]:
def calc_consistency(s: pd.Series):
    if s.count() == 0:
        return np.nan
    
    max_points = max(s)
    if max_points == 0:
        return np.nan
    
    return np.mean(s/max(s))*100

def calc_stats(df: pd.DataFrame, game_week: int = None):
    team_id = df['Player Team ID'].iloc[0]
    
    if not game_week is None:
        df = df[df['Game Week'] <= game_week]
    
    s={'Total Points': df['Game Total Points'].sum(),
       'Total Points Consistency': calc_consistency(df['Game Total Points']),
       'Player Team ID': team_id}
    
    return pd.Series(s)
    
players_history_fixtures = players_history[['Game Total Points', 'Game Minutes Played', 'Game Cost']].reset_index() \
    .merge(fixtures, left_on='Fixture ID', right_index=True).set_index('Player ID')
players_history_fixtures = players_history_fixtures.merge(players[['Player Team ID', 'Field Position']], left_index=True, right_index=True)
players_history_stats = players_history_fixtures.groupby(['Player ID']).apply(lambda df: calc_stats(df))
player_teams_stats = player_teams.reset_index().merge(players_history_stats[['Total Points Consistency']], left_on='Player ID', right_on='Player ID')\
    .sort_values(['Total Points Consistency'], ascending=False).set_index('Player ID')
player_teams_stats['Points Per Cost'] = player_teams['Total Points']/player_teams['Current Cost']

# Add the total points from the last season to the player stats so it can be used for the expected point calculation at the beginning of the season.
players_history_last_season = players_history_past.reset_index()[['Player ID', 'Season', 'Season Total Points']]
players_history_last_season = players_history_last_season[players_history_last_season['Season'] == last_season].rename(columns={'Season Total Points': 'Last Season Total Points'})
player_teams_stats = player_teams_stats.merge(players_history_last_season.set_index('Player ID'), left_index=True, right_index=True, how='left')

player_teams_stats = dd.reorder(player_teams_stats)
# Uncomment to view data: dd.display(player_teams_stats, head=15)

## Fixture metrics
In order to calculate relative strengths of the teams, we aggregate the points that the team has earned so far. We later can use this information to adjust the expected points for each player.

In [ ]:
team_points = player_teams_stats.reset_index().groupby(['Player Team ID', 'Team Name', 'Team Short Name'])[['Total Points']].sum().sort_values('Total Points', ascending=False)
team_points = team_points.reset_index().set_index('Player Team ID')
# Uncomment to view data:  dd.display(team_points, excel_file='team_points.xlsx', head=50)

In [ ]:
fixture_teams = fixture_teams.merge(team_points[['Total Points']].rename(columns={'Total Points': 'Fixture Strength Home'}), left_on='Home Team ID', right_on='Player Team ID')
fixture_teams = fixture_teams.merge(team_points[['Total Points']].rename(columns={'Total Points': 'Fixture Strength Away'}), left_on='Away Team ID', right_on='Player Team ID')
fixture_teams['Rel. Fixture Strength Home'] = fixture_teams['Fixture Strength Home']/fixture_teams['Fixture Strength Away']
fixture_teams['Rel. Fixture Strength Away'] = fixture_teams['Fixture Strength Away']/fixture_teams['Fixture Strength Home']
fixture_teams['Fixture Short Name'] = fixture_teams['Team Short Name Home']+'-'+fixture_teams['Team Short Name Away']
fixture_teams = dd.reorder(fixture_teams)
# Uncomment to view data:  dd.display(fixture_teams.sort_values(['Game Week']), excel_file='fixture_teams.xlsx', head=50)

Calculates the strength metrics for each fixture from the team's point of view.

In [ ]:
# Unfold data frame so that there a two rows for each fixture.
team_fixture_strength = pd.melt(fixture_teams[['Fixture ID', 'Game Week', 'Finished', 'Kick Off Time', 'Home Team ID', 'Team Name Home', 'Away Team ID', 'Fixture Short Name', 'Fixture Strength Home', 'Fixture Strength Away', 'Rel. Fixture Strength Home', 'Rel. Fixture Strength Away', 'Home Team Difficulty', 'Away Team Difficulty']],\
        id_vars=['Fixture ID', 'Game Week', 'Finished', 'Kick Off Time', 'Fixture Strength Home', 'Fixture Strength Away', 'Rel. Fixture Strength Home', 'Rel. Fixture Strength Away', 'Home Team Difficulty', 'Away Team Difficulty', 'Fixture Short Name'], 
        value_vars=['Home Team ID', 'Away Team ID'])\
    .rename(columns={'variable': 'Variable', 'value': 'Value'})\
    .sort_values(['Game Week'])

# Get the correct strength based on whether the team is the home or away team.
team_fixture_strength['Rel. Fixture Strength'] = team_fixture_strength.apply(lambda row: row['Rel. Fixture Strength Home'] if row['Variable'] == 'Home Team ID' else row['Rel. Fixture Strength Away'], axis=1)
team_fixture_strength['Team Difficulty'] = team_fixture_strength.apply(lambda row: row['Home Team Difficulty'] if row['Variable'] == 'Home Team ID' else row['Away Team Difficulty'], axis=1)
team_fixture_strength = team_fixture_strength.rename(columns={'Value': 'Team ID'}).drop('Variable', axis=1)

team_fixture_strength = team_fixture_strength.merge(teams, left_on='Team ID', right_on='Team ID')
team_fixture_strength = dd.reorder(team_fixture_strength)
# Uncomment to view data for a specific game week: dd.display(team_fixture_strength[team_fixture_strength['Game Week'] == 5])
# Uncomment to view data: dd.display(team_fixture_strength, excel_file='team_fixture_strength.xlsx')

# Explore players based on expected points for different time horizons
The forecast of the points earned by each player for a specific time horizon is based on the average points earned over the past fixtures this season and then adjusting it for the difficulty of the upcoming fixtures over specific time horizon and likelihood of playing. It is calculated for every player as:

$$ep_{th} = \frac{tp}{gw}\sum_{i=1}^{t} p_i\frac{\sum_{i=1}^{th} s_i}{s_{gw}} $$

where

* $ep_{th}$: Expected points for a players over a specific time horizon, e.g. next game week, next five game weeks, remaining game weeks of the current season half and remaining game weeks of the current season
* $tp$: Total points earned by the player this season so far
* $gw$: Number of game weeks completed so far
* $s_i$: Relative strenth of a specific upcoming fixture for the team of the player. This value is usally just over 1 if the player's team is relatively to stronger and less than 1 if it is weaker.
* $s_{gw}$: Relative strenth of fixtures this season so far for the team of the player
* $p_i$: Chance of player playing in an upcoming fixture

This approach the following limitations:
* Expected points are solely predicted based on the past performance and no other information such as recent news is taken into account.
* All past performance this season is considered equal, regardless of how long ago it was. I.e. there is no recency bias.
* Past perforamnce does not take into account why a player has not earned any points during a game. It matters whether this due to manager selection or due to inhjury.
* The relative strengths are calculated based on many assumptions and the full meaning of the raw strength numbers published by FPL is not clear.
* The adjustment for the chance of the player playing really should be made on a per game basis as opposed to the overall expected points (but given the accuracy of the chance of playing this is probably a minor issue).

The next step is to backtest the predictions.

## Calculate relative fixture strengths
Calculates a relative fixtures strengths $s$ for each team for the following time horizons:
* all played game weeks
* the next game week (mainly for Free Hit)
* the next five game weeks (mainly for individuals transfers)
* the remaining game weeks of the current season half (mainly for Wildcard)
* the remaining game weeks of the current season

The relative strength is a factor around 1 and is used in the expected point prediction below to adjust the predicted points based on the relative strengths of the upcoming game weeks. The simple idea here is that the relative strong a team is the higher the expected points. A value above 1 indicates that the player's team is relatively stronger and a value below 1 indicates that the team is relatively weaker. 

In [ ]:
# Get current game week. Only when all fixtures are completed is this number incremented.
current_gw = max(1, team_fixture_strength[team_fixture_strength['Finished'] == False]['Game Week'].min())
total_game_weeks = team_fixture_strength['Game Week'].max()

def value_or_default(value, default = np.nan) -> str:
    return default if value is None or isinstance(value, str) and value == '' or not isinstance(value, str) and np.isnan(value) else value
         
def calc_future_strengths(df: pd.DataFrame, next_gw: int) -> dict:    
    def calc_historic_strength(df: pd.DataFrame, next_gw: int) -> int:
        return df[df['Game Week'] < next_gw]['Rel. Fixture Strength'].mean()

    def calc_future_strength(df: pd.DataFrame, next_gw: int, gw_count: int) -> int:
        return df[(df['Game Week'] >= next_gw) & (df['Game Week'] < next_gw+gw_count)]['Rel. Fixture Strength'].mean()

    def get_fixtures(df: pd.DataFrame, next_gw: int, gw_count: int) -> str:
        df['Fixture Short Name Difficulty'] = df['Fixture Short Name']+' ('+df['Team Difficulty'].astype('str')+')'
        return value_or_default(df[(df['Game Week'] >= next_gw) & (df['Game Week'] < next_gw+gw_count)]['Fixture Short Name Difficulty'].str.cat(sep=', '))

    half_game_weeks = int(total_game_weeks/2)
    remain_half_game_weeks = half_game_weeks-next_gw+1 if next_gw <= half_game_weeks else total_game_weeks-next_gw+1
    remain_game_weeks = total_game_weeks-next_gw+1
    
    # Determines the number of game weeks to look ahead but not beyond end of season
    s = {}
    s['Game Week'] = next_gw
    s['Next GW'] = min(1, total_game_weeks-next_gw+1)
    s['Next 5 GWs'] = min(5, total_game_weeks-next_gw+1)
    s['GWs To Half'] = min(int(total_game_weeks/2), remain_half_game_weeks)
    s['GWs To End'] = remain_game_weeks
    s['Rel. Fixture Strength Finished GWs'] = calc_historic_strength(df, next_gw)
    s={**s, **dict(('Rel. Fixture Strength '+gw, 
                    calc_future_strength(df, next_gw, s[gw])) for gw in next_gws)}
    s={**s, **dict(('Fixtures '+gw, 
        get_fixtures(df, next_gw, s[gw])) for gw in next_gws[:2])}
    return pd.Series(s)


In [ ]:
# Unit tests the calc_future_strengths function.
from pandas.util.testing import assert_frame_equal

def test_calc_future_strengths_first_gw():
    team_strengths = pd.read_csv('tests/unit/test_calc_future_strengths.csv')\
        .groupby(['Team ID'])\
        .apply(lambda df: calc_future_strengths(df, next_gw=1))\
        .reset_index()
    
    assert_frame_equal(pd.read_csv('tests/unit/assert_calc_future_strengths_first_gw.csv'), team_strengths)

def test_calc_future_strengths_missing_fixture():
    team_strengths = pd.read_csv('tests/unit/test_calc_future_strengths.csv')\
        .groupby(['Team ID'])\
        .apply(lambda df: calc_future_strengths(df, next_gw=18))\
        .reset_index()
    
    assert_frame_equal(pd.read_csv('tests/unit/assert_calc_future_strengths_missing_fixture.csv'), team_strengths)

def test_calc_future_strengths_half_season():
    team_strengths = pd.read_csv('tests/unit/test_calc_future_strengths.csv')\
        .groupby(['Team ID'])\
        .apply(lambda df: calc_future_strengths(df, next_gw=20))\
        .reset_index()
    
    assert_frame_equal(pd.read_csv('tests/unit/assert_calc_future_strengths_half_season.csv'), team_strengths)

def test_calc_future_strengths_last_gw():
    team_strengths = pd.read_csv('tests/unit/test_calc_future_strengths.csv')\
        .groupby(['Team ID'])\
        .apply(lambda df: calc_future_strengths(df, next_gw=38))\
        .reset_index()
    
    assert_frame_equal(pd.read_csv('tests/unit/assert_calc_future_strengths_last_gw.csv'), team_strengths)
    
test_calc_future_strengths_first_gw()
test_calc_future_strengths_missing_fixture()
test_calc_future_strengths_half_season()
test_calc_future_strengths_last_gw()

## Transfer relative fixture strengths from fixtures to players
This section joins the fixture strengths data set with the player data set so that expected points can be calculated on a player basis. For this it joins two sets: one historical and one future set. Reason for this is that for completed matches, we need it to use consider that the team that player actually played for, while for future games we can assume that the player will play for the same team than he is currently in.

In [ ]:
players_history_fixture_team_strengths = players_history_fixtures.reset_index().merge(team_fixture_strength[['Team ID', 'Fixture ID', 'Team Short Name', 'Team Name', 'Rel. Fixture Strength', 'Fixture Short Name', 'Team Difficulty']],
                                                  left_on=['Player Team ID', 'Fixture ID'], right_on=['Team ID', 'Fixture ID'], how='left')


In [ ]:
# Estimates the chance that a player is available for the future game weeks.
def est_chance_avail(df: pd.DataFrame)-> pd.Series:
    df = df.sort_values('Game Week')
    chance_avail = df['Chance Avail Next GW']
    
    if chance_avail.shape[0] == 0:
        return chance_avail
    
    if chance_avail.iloc[0] > 0 and chance_avail.iloc[0] < 1: # If the chance available is not 0 or 1 then assume that the following game week the chance is 1.
        chance_avail.iloc[1:] = 1
        
    return chance_avail

team_future_fixture_strengths = team_fixture_strength[['Team ID', 'Fixture ID', 'Team Short Name', 'Team Name', 'Rel. Fixture Strength', 'Fixture Short Name', 'Team Difficulty', 'Game Week']][team_fixture_strength['Fixture ID'].isin(players_history_fixture_team_strengths['Fixture ID'].values) == False]
players_future_fixture_team_strengths = player_teams[['Player Team ID', 'ICT Index', 'Chance Avail Next GW']].reset_index() \
    .merge(team_future_fixture_strengths, left_on=['Player Team ID'], right_on=['Team ID'])

chance_avail = players_future_fixture_team_strengths[['Player ID', 'Chance Avail Next GW', 'Game Week']]\
    .groupby('Player ID').apply(lambda df: est_chance_avail(df)).droplevel('Player ID').rename('Chance Avail')

players_future_fixture_team_strengths = players_future_fixture_team_strengths.drop(columns=['Chance Avail Next GW'])
players_future_fixture_team_strengths = pd.merge(players_future_fixture_team_strengths, pd.DataFrame(chance_avail), left_index=True, right_index=True)

In [ ]:
players_fixture_team_strengths = pd.concat([players_history_fixture_team_strengths, players_future_fixture_team_strengths], sort=False)

## Calculates the expected points for the following time horizons
Calculates the expected points for the following time horizons:
* the next game week (mainly for Free Hit)
* the next five game weeks (mainly for individuals transfers)
* the remaining game weeks of the current season half (mainly for Wildcard)
* the remaining game weeks of the current season

The expected return simply calculates the average number of points achieved of the past game week and extrapolates them over the next x game weeks. This number is then adjusted by the ratio of the relative strength of upcoming fixtures and the relative strength of the past ones and for each player by the propability of the player being available to play.

In [ ]:
def calc_expected_points_gw(row: pd.Series, post_fix: str, adjust_for_strength) -> pd.Series:
    if row['Game Weeks Played'] == 0: # If the season has not started, calculate the average based on the last season.
        return np.nan # We need to find a way of estimating expected points for game week 0 based on previous seasons.
    else: # Otherwise calculate the average points per game week.
        expected_points = row['Total Points']/row['Game Weeks Played']*row[post_fix]

        if adjust_for_strength:
            expected_points *= max(0, row['Rel. Fixture Strength '+post_fix])/row['Rel. Fixture Strength Finished GWs']

    expected_points *= max(0, row['Avg Chance Avail '+post_fix])/100

    return expected_points
 
def calc_chance_avail_gw(df: pd.DataFrame, next_gw: int, gw_count: int) -> float:
    df = df[['Game Week', 'Chance Avail']]
    df = df[(df['Game Week'] >= next_gw) & (df['Game Week'] < next_gw+gw_count)]
    return value_or_default(df['Chance Avail'].fillna(100).mean(), 0)

def last_or_default(series: pd.Series, default=np.nan):
    if series is None:
        return default
    
    series = series[~series.isnull()]
    if series.shape[0] == 0:
        return default
    
    return series.iloc[-1]

def calc_expected_points(df: pd.DataFrame, next_gw: int, adjust_for_strength = True) -> pd.Series:
    df = df.sort_values(by='Game Week')
    row = calc_future_strengths(df, next_gw)
    row['Game Week'] = next_gw
    row['Game Weeks Played'] = df['Game Week'][(df['Game Week'] < next_gw) & (df['Game Minutes Played'] > 0)].count()
    row['Game Total Points'] = df['Game Total Points'][df['Game Week'] == next_gw].pipe(last_or_default) # Actual points in the next game week. Required for backtesting.
    row['Total Points'] = df['Game Total Points'][df['Game Week'] < next_gw].sum()
    row['Current Cost'] = df['Game Cost'][df['Game Week'] <= next_gw].pipe(last_or_default)
    row['Total Points Consistency'] = calc_consistency(df['Game Total Points'][df['Game Week'] < next_gw])
    avg_chance_avail = pd.Series(dict(('Avg Chance Avail '+gw,calc_chance_avail_gw(df, next_gw, row[gw])) for gw in next_gws))
    row = pd.concat([row, avg_chance_avail], sort=False)
    s = pd.Series(dict(('Expected Points '+gw, calc_expected_points_gw(row, gw, adjust_for_strength)) for gw in next_gws))
    row = pd.concat([row, s], sort=False)
    return row 

In [ ]:
# Unit tests the calc_expected_points function.
def test_calc_expected_points_first_gw():
    players_fixture_team_strengths = pd.read_csv('tests/unit/test_calc_expected_points.csv')\
        .groupby(['Player ID'])\
        .apply(lambda df: calc_expected_points(df, next_gw=1))\
        .reset_index()
    
    assert_frame_equal(pd.read_csv('tests/unit/assert_calc_expected_points_first_gw.csv'), players_fixture_team_strengths, check_dtype=False)


def test_calc_expected_points_missing_fixture():
    players_fixture_team_strengths = pd.read_csv('tests/unit/test_calc_expected_points.csv')\
        .groupby(['Player ID'])\
        .apply(lambda df: calc_expected_points(df, next_gw=18))\
        .reset_index()
    
    assert_frame_equal(pd.read_csv('tests/unit/assert_calc_expected_points_missing_fixture.csv'), players_fixture_team_strengths, check_dtype=False)

test_calc_expected_points_first_gw()
test_calc_expected_points_missing_fixture()

In [ ]:
player_team_expected_points = players_fixture_team_strengths.groupby('Player ID').apply(lambda df: calc_expected_points(df, next_gw=current_gw))
player_team_expected_points = player_team_expected_points.merge(player_teams[['Name', 'Field Position ID', 'Field Position', 'News And Date', 'Player Team ID', 'First Name', 'Last Name', 'ICT Index', 'Team Short Name', 'Name and Short Team', 'Minutes Percent']],
                                           left_index=True, right_index=True)
player_team_expected_points = player_team_expected_points[player_team_expected_points['Next GW'].isnull() == False] # Remove entries for which there is no known next game week.
player_team_expected_points_active = player_team_expected_points[player_team_expected_points['Minutes Percent'] > 50] # Remove entries with less than 50% minutes played.

## Visualise players' earned points per cost and the expected points

In [ ]:
def player_strength_by_horizon(player_expected_points: pd.DataFrame, horizon: str, dd: pd.DataFrame):
    """
    Returns a plotly chart with expected points as the y-axis and cost on the x-axis for a specific time horizon. This chart can be displayed in the Jupyter notebook.

    Args:
        player_expected_points: The data frame with data to chart.
        horizon: The time horizon of the chart, e.g. Next GW, Next 5 GWs, etc.
        dd: The data dictionary to use for formatting.

    Returns:
        The plotly chart.
    """
    player_expected_points = player_expected_points[
        ['Name and Short Team', 'Field Position', 'Current Cost', 'Total Points', 'Minutes Percent', 'News And Date', 'Fixtures Next 5 GWs', 'ICT Index', 'Total Points Consistency']
        + ['Expected Points ' + next_gw for next_gw in next_gws]
        + ['Rel. Fixture Strength ' + next_gw for next_gw in next_gws]
        + (['In Team?'] if 'In Team?' in player_expected_points.columns.values else [])]
    player_expected_points_formatted = dd.format(player_expected_points)
    player_expected_points['Label'] = player_expected_points_formatted['Name and Short Team'] \
                                      + ', ' + player_expected_points_formatted['Field Position'] \
                                      + ', Cost: ' + player_expected_points_formatted['Current Cost'] \
                                      + ', Total Points: ' + player_expected_points_formatted['Total Points'] \
                                      + '<br>Minutes Percent: ' + player_expected_points_formatted['Minutes Percent'] \
                                      + ', Consistency: ' + player_expected_points_formatted['Total Points Consistency'] \
                                      + ', ICT: ' + player_expected_points_formatted['ICT Index'] \
                                      + ', Rel. Strength: ' + player_expected_points_formatted['Rel. Fixture Strength '+horizon] \
                                      + '<br>Next: ' + player_expected_points_formatted['Fixtures Next 5 GWs'].astype('str') \
                                      + '<br>News: ' + player_expected_points_formatted['News And Date'].astype('str')

    colors = {'GK': 'rgba(31, 119, 180, 1)', 'DEF': 'rgba(255, 127, 14, 1)', 'MID': 'rgba(44, 160, 44, 1)', 'FWD': 'rgba(214, 39, 40, 1)'}

    data = []
    if 'In Team?' in player_expected_points.columns.values:
        data += [Scatter(**{
            'x': player_expected_points[player_expected_points['In Team?'] == True]['Current Cost'],
            'y': player_expected_points[player_expected_points['In Team?'] == True]['Expected Points ' + horizon],
            'mode': 'markers',
            'marker': {
                'size': 15,
                'color': 'white',
                'line': {'width': 1},
            },
            'name': 'In Team',
            'text': player_expected_points[player_expected_points['In Team?'] == True]['Label']})]

    data += [Scatter(**{
        'x': player_expected_points[player_expected_points['Field Position'] == position]['Current Cost'],
        'y': player_expected_points[player_expected_points['Field Position'] == position]['Expected Points ' + horizon],
        'name': position,
        'mode': 'markers',
        'marker': {
            'color': colors[position],
        },
        'text': player_expected_points[player_expected_points['Field Position'] == position]['Label']
    }) for position in position_by_type.values()]

    return (py.iplot(
        {
            'data': data,
            'layout': {
                'xaxis': {'title': 'Current Cost (lower is better)', 'showspikes': True},
                'yaxis': {'title': f'Expected Points {horizon} (higher is better)', 'showspikes': True},
                'hovermode': 'closest'
            }
        }
    ))

In [ ]:
_ = interact(player_strength_by_horizon, player_expected_points=fixed(player_team_expected_points_active), dd=fixed(dd), horizon=widgets.ToggleButtons(description='Horizon: ', options=next_gws))

# Load user team data
This section loads the data of the user's team. **Note this requires your user credentials to be saved in fpl_credentials.csv in the same directory as this notebook.**

In [ ]:
creds_file = 'fpl_credentials.csv'
if not 'fpl_email' in globals(): fpl_email = ''
user_widget, pass_widget = [None, None]
if os.path.exists(creds_file):
    fpl_cred = pd.read_csv('fpl_credentials.csv')
    fpl = FPLPandas(**fpl_cred.iloc[1].to_dict())
else:
    user_widget = widgets.Text(value=fpl_email, placeholder='Enter FPL email address', description='Email')
    pass_widget = widgets.Text(value='', placeholder='Enter FPL password', description='Password')
    display(widgets.HBox([user_widget, pass_widget]))

In [ ]:
if not user_widget is None and not pass_widget is None:
    fpl_email = user_widget.value
    fpl = FPLPandas(email=fpl_email, password=pass_widget.value)

try:
    user_team_raw, _, user_trans_info_raw = fpl.get_user_team()
except Exception as e:
    print(e)
    
if not pass_widget is None:
    pass_widget.value = ''

In [ ]:
user_team = dd.remap(user_team_raw, data_set='player')
user_team['In Team?'] = True
user_team['Selling Price'] = user_team['Selling Price']/10
user_team['Purchase Price'] = user_team['Purchase Price']/10
user_team['Selected?'] = user_team['Team Position'].map(lambda x: x <= 11)
user_team.index = user_team.index.rename('Player ID')
user_trans_info = user_trans_info_raw.loc[0]

## Current team

In [ ]:
player_user_team = user_team.merge(player_team_expected_points, left_on='Player ID', right_on='Player ID', how='left')
display_team(player_user_team, dd)

In [ ]:
_ = interact(player_strength_by_horizon, player_expected_points=fixed(user_team.merge(player_team_expected_points_active, left_on='Player ID', right_on='Player ID', how='outer')),
             dd=fixed(dd), horizon=widgets.ToggleButtons(description='Horizon: ', options=next_gws))

# Get best team for wildcard or season start
You can use the code below to get the best team for a wildcared or at the start of the season. It uses the [PuLP linear optimiser](https://pythonhosted.org/PuLP/) to find the team combination within the current money available with the highest total expected points of the over the next five gameweeks.

In [ ]:
total_budget = (user_trans_info['bank']/10+player_user_team['Current Cost'].sum())
total_budget

In [ ]:
player_team_optimal = get_optimal_squad(player_team_expected_points_active, 
                                        optimise_team_on='Expected Points Next 5 GWs',
                                        optimise_sel_on='Expected Points Next GW', 
                                        formation='2-5-5-3', 
                                        budget=total_budget)\
    .sort_values(['Field Position ID'])
player_team_optimal = dd.reorder(player_team_optimal)
display_team(player_team_optimal, dd)

# Recommend selection or next GW and transfer for next 5 GWs
Use this section to get a recommendation on what players to select to optimise the expected points of your team and to improve it by making transfers. You need to have provided your FPL credentials for this to work.

It uses the PuLP linear optimiser to find the team combination within the current money available with the highest total expected points of the over the next five gameweeks while taking your current team into account for a user defined number of transfers. Note that when executing more than one transfer on the FPL website, 4 points will be deducted from your balance for every transfer.

It uses the same PuLP linear optimiser to find the selection with the highest expected points for the next game week.

## Recommended team

In [ ]:
# Gets the cost and player ID of the second goal keeper so that the optimiser does not recommend his replacement.
second_gk = player_user_team[player_user_team['Field Position'] == 'GK'].sort_values('Expected Points Next GW')[['Current Cost']].iloc[0]
second_gk_cost = second_gk.values[0]
second_gk_id = second_gk.name

player_team_expected_points_user = user_team.merge(player_team_expected_points_active, left_on='Player ID', right_on='Player ID', how='right')
player_team_optimal = (get_optimal_squad(player_team_expected_points_user, 
                                        optimise_team_on='Expected Points Next 5 GWs',
                                        optimise_sel_on='Expected Points Next GW', 
                                        formation='1-5-5-3', # Not 2-5-5-3 if we want to avoid the transfer of the second goal keeper recommended.
                                        budget=total_budget-second_gk_cost, # Not just total_budget if we want to avoid the transfer of the second goal keeper recommended.
                                        recommend=2) # If set to 0, the optimiser will still recommend a team selection that maximises the expected points.\
    .sort_values(['Field Position ID']))
player_team_optimal = dd.reorder(player_team_optimal)
display_team(player_team_optimal, dd, in_team=True)

In [ ]:
player_team_removed = player_user_team[(player_user_team['In Team?'] == True) 
                                       & (player_user_team.index.isin(player_team_optimal.index.values) == False)
                                      & (player_user_team.index.isin([second_gk_id]) == False)]
dd.display(player_team_removed[['Name', 'Current Cost', 'Field Position', 'Captain?', 'Vice Captain?', 'Minutes Percent', 'News And Date', 'Expected Points Next GW', 'Expected Points Next 5 GWs', 'Total Points Consistency']],
           index=False, footer=False, descriptions=False)

# Select a good week to play free hit
The idea is that there are game weeks when there are fixtures with large differences in team strengths than in others. This is the case when stronger teams mainly play weaker teams. The assumptions is that these game weeks have higher potential for earning points for the stronger teams. The list below shows the game weeks sorted by the average relative fixture strength (of the four fixtures with the highest relative fixture strength).

In [ ]:
(team_fixture_strength.groupby('Game Week')['Rel. Fixture Strength']\
 .apply(lambda x: x.nlargest(4)).sum(level=0)/4)\
 .sort_values(ascending=False).to_frame().head(10)

# How predictive is the ICT index?
The Fantasy Premier League website says:
>The ICT Index is a football statistical index developed specifically to assess a player as an FPL asset. It uses match event data to generate a single score for three key areas – Influence, Creativity and Threat. These figures then combine to create an individual’s ICT Index score. It condenses more than 40 match event statistics into four distinct scores. These offer a view on player performance for factors that are known to produce FPL points. See https://fantasy.premierleague.com/help

So as the season progresses the chart should converge to a line if the index is indeed predictive.

In [ ]:
# Calculate a fitted straight line to show what the ideal would look like
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split


# Count the number of players
player_count = player_teams.shape[0]

# Split the data into training/testing sets
total_points = np.reshape(player_teams['Total Points'].values, (player_teams.shape[0], 1))
itc_index = np.reshape(player_teams['ICT Index'].values, (player_teams.shape[0], 1))
total_points_train, total_points_test, itc_index_train, itc_index_test = train_test_split(total_points, itc_index, test_size = 0.4, random_state = 1)

# Train linear regression model
regr = LinearRegression()
_ = regr.fit(total_points_train, itc_index_train)

# Make predictions using the testing set
itc_index_pred = regr.predict(total_points_test)

# The coefficients
print(f'Coefficients: {regr.coef_[0][0]:.2f} ')
# The mean squared error
print(f'Mean squared error: {np.mean((regr.predict(itc_index_test) - itc_index_pred) ** 2):.2f}')
# Explained variance score: 1 is perfect prediction
print(f'Variance score: {regr.score(total_points_test, itc_index_pred):.2f}')

In [ ]:
fig = {
    'data': [
        {'x': player_teams['Total Points'], 'y': player_teams['ICT Index'], 'text': player_teams['Name and Short Team'], 'mode': 'markers', 'name': 'Data'},
        {'x': np.reshape(total_points_test,  (1, total_points_test.shape[0]))[0], 'y': np.reshape(regr.predict(total_points_test), (1, total_points_test.shape[0]))[0], 'mode': 'lines', 'name': 'Ideal (fully predictive)'}
    ],
    'layout': {
        'xaxis': {'title': 'Total Point'},
        'yaxis': {'title': 'ICT Index'}
    }
}
py.iplot(fig)

# Back test the expected points
The basic idea of testing the predictions is to look at each past game week, predict the expected points for the game week (both adjusted for relative team strengths and not adjusted), optimise the team based on the expected points and then calculate the total expected points for the optimised team (only for the selected player). For validation, we calculate the actual points of the players of the optimised team. We also calculate the points of the dream team, i.e. the total points of the team with highest actual points for each game week.

In [ ]:
from sklearn.metrics import mean_absolute_error
import timeit

def get_optimal_team_exp(player_team_exp_gw: pd.DataFrame) -> pd.DataFrame:
    global player_team_optimal
    player_team_optimal = get_optimal_squad(player_team_exp_gw, 
                                    optimise_team_on='Expected Points Next GW',
                                    optimise_sel_on='Expected Points Next GW', 
                                    formation='2-5-5-3', 
                                    budget=100)
    return player_team_optimal[['Total Points', 'Game Total Points', 'Expected Points Next GW', 'Point Factor', 'Selected?']]
 
    
def get_optimal_team_act(players_history_fixtures_gw: pd.DataFrame) -> pd.DataFrame:
    player_team_optimal_act = get_optimal_squad(players_history_fixtures_gw, 
                                    optimise_team_on='Game Total Points',
                                    optimise_sel_on='Game Total Points', 
                                    formation='2-5-5-3', 
                                    budget=100)
    return player_team_optimal_act[['Game Total Points', 'Selected?']]  


def calc_team_points(player_team: pd.DataFrame, points_col: str) -> float:
    player_team = player_team[player_team['Selected?'] == True]
    player_team['Points'] = player_team[points_col]
    
    if 'Point Factor' in player_team.columns.values:
        player_team['Points'] *= player_team['Point Factor']

    return player_team['Points'].sum()


def back_test_gw(gw: int, players_fixture_team_strengths: pd.DataFrame, player_teams: pd.DataFrame, team_fixture_strength: pd.DataFrame) -> dict:   
    players_fixture_team_strengths = players_fixture_team_strengths.copy()
    players_fixture_team_strengths['News And Date'] = None # Unfortunately, we don't have historic news information.
    players_history_fixtures_gw = players_fixture_team_strengths[players_fixture_team_strengths['Game Week'] == gw].rename(columns={'Game Cost': 'Current Cost'})

    player_team_exp_gw = players_fixture_team_strengths.groupby('Player ID').apply(lambda df: calc_expected_points(df, next_gw=gw))
    player_team_exp_gw = player_team_exp_gw[~player_team_exp_gw['Expected Points Next GW'].isnull()]
    player_team_exp_gw = player_team_exp_gw.merge(player_teams[['Name', 'Field Position ID', 'Field Position', 'News And Date', 'Player Team ID', 'First Name', 'Last Name', 'ICT Index', 'Team Short Name', 'Name and Short Team', 'Minutes Percent']],
                                           left_index=True, right_index=True)
    player_team_exp_gw = player_team_exp_gw[(player_team_exp_gw['Game Total Points'] > 0) & (player_team_exp_gw['Minutes Percent'] > 80)] 

    player_team_optimal_exp = get_optimal_team_exp(player_team_exp_gw)
    #player_team_optimal_exp_adj = get_optimal_team_exp(player_team_exp_gw)    
    player_team_optimal_act = get_optimal_team_act(players_history_fixtures_gw)
        
    return {'Game Week': gw, 
            'Actual Points Dream Team': calc_team_points(player_team_optimal_act, 'Game Total Points'),
            'Expected Points (adjusted)': calc_team_points(player_team_optimal_exp, 'Expected Points Next GW'), 
            'Actual Points (adjusted)': calc_team_points(player_team_optimal_exp, 'Game Total Points')}
    

backtest_results = pd.DataFrame()
for gw in log_progress(range(2, current_gw), name='Game weeks'):
    backtest_results = backtest_results.append(back_test_gw(gw, players_fixture_team_strengths, player_teams, team_fixture_strength), ignore_index=True)

py.iplot([{'x': backtest_results['Game Week'], 
           'y':  backtest_results[col], 'name': col} for col in ('Actual Points Dream Team', 'Expected Points (adjusted)', 'Actual Points (adjusted)')])